In [11]:
import requests

In [12]:
response = requests.get("http://127.0.0.1:5000/country_test_done")
print(response.status_code)

200


In [18]:
# print(response.json())

In [20]:
response.json()['data'].keys()

dict_keys(['country', 'country_code', 'year_week', 'level', 'region', 'region_name', 'new_cases', 'tests_done', 'population', 'testing_rate', 'positivity_rate', 'testing_data_source'])

In [21]:
# import json

# def jprint(obj):
#     # create a formatted string of the Python JSON object
#     text = json.dumps(obj, sort_keys=True, indent=4)
#     print(text)

# jprint(response.json())

In [30]:
query = {'country':'DK'}

In [31]:
response = requests.get('http://127.0.0.1:5000/country_test_done', params=query)

In [114]:
test = pd.read_csv('./Downloads/covid_testing_data.csv')

country_cond = test['country'].isin(["Denmark", "Germany", "Romania", "Spain", "Sweden"])
level_cond = test['level'] == 'national'

test = (
    test[country_cond & level_cond]
    .drop(['level', 'region_name'], axis = 1)
)

In [115]:
list_year_week = test['year_week'].str.split('-W')

test['year'] = [elem[0] for elem in list_year_week]
test['week'] = [int(elem[1]) for elem in list_year_week]
test['month'] = (test['week'] / 4).astype('int') + 1

In [116]:
agg_df = test\
.groupby(['year','month', 'country', 'country_code'])\
.agg({'new_cases': 'sum', 'tests_done': 'sum', 'population': 'mean'})\
.reset_index()

In [117]:
agg_df['testing_rate'] = agg_df['tests_done'] / agg_df['population'] * 100
agg_df['positivity_rate'] = agg_df['new_cases'] / agg_df['tests_done'] * 100

In [118]:
agg_df

,year,month,country,country_code,new_cases,tests_done,population,testing_rate,positivity_rate
0,2020,2,Denmark,DK,0,11,5806081.0,0.000189,0.000000
1,2020,2,Sweden,SE,1,153,10230185.0,0.001496,0.653595
2,2020,3,Denmark,DK,875,6110,5806081.0,0.105234,14.320786
3,2020,3,Germany,DE,4709,252173,83019213.0,0.303753,1.867369
4,2020,3,Sweden,SE,1062,14071,10230185.0,0.137544,7.547438
...,...,...,...,...,...,...,...,...,...
59,2021,1,Denmark,DK,14014,566404,5806081.0,9.755358,2.474206
60,2021,1,Germany,DE,145511,789680,83019213.0,0.951202,18.426578
61,2021,1,Romania,RO,32842,122035,19414458.0,0.628578,26.911951
62,2021,1,Spain,ES,152938,988746,46937060.0,2.106536,15.467875


In [82]:
test.groupby(['country', ])

<ipython-input-82-cdb5a93cbe99>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['testing_rate'] = a['testing_rate'] / 100_000 * 100
<ipython-input-82-cdb5a93cbe99>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['new'] = a['tests_done'] / a['population']


,country,country_code,year_week,region,new_cases,tests_done,population,testing_rate,positivity_rate,testing_data_source,new
430,Denmark,DK,2020-W50,DK,19155,685789,5806081.0,11.811564,2.793133,TESSy,0.118116
678,Germany,DE,2020-W50,DE,153424,1297303,83019213.0,1.562654,11.826381,TESSy,0.015627
2824,Romania,RO,2020-W50,RO,42759,170289,19414458.0,0.877125,25.109667,TESSy,0.008771
2961,Spain,ES,2020-W50,ES,49556,785052,46937060.0,1.672563,6.312448,TESSy,0.016726
3278,Sweden,SE,2020-W50,SE,43639,270944,10230185.0,2.648476,16.106280,TESSy,0.026485


In [66]:
test.sort_values('year_week', ascending=False)

,country,country_code,year_week,level,region,region_name,new_cases,tests_done,population,testing_rate,positivity_rate,testing_data_source
3282,Sweden,SE,2021-W01,national,SE,Sweden,39485,201011,10230185.0,1964.881378,19.643204,TESSy
869,Ireland,IE,2021-W01,national,IE,Ireland,45726,173845,4904240.0,3544.789815,26.302741,TESSy
983,Italy,IT,2021-W01,subnational,ITC4,Lombardia,16313,138017,10060574.0,1371.860095,11.819558,TESSy
966,Italy,IT,2021-W01,subnational,ITC3,Liguria,2491,24780,1550640.0,1598.049837,10.052462,TESSy
949,Italy,IT,2021-W01,subnational,ITC2,Valle D'aosta/vallee D'aoste,173,1537,125666.0,1223.083412,11.255693,TESSy
...,...,...,...,...,...,...,...,...,...,...,...,...
541,Finland,FI,2020-W04,national,FI,Finland,0,21,5517919.0,0.380578,0.000000,Survey
540,Finland,FI,2020-W03,national,FI,Finland,0,12,5517919.0,0.217473,0.000000,Survey
539,Finland,FI,2020-W02,national,FI,Finland,0,14,5517919.0,0.253719,0.000000,Survey
131,Bulgaria,BG,2020-W01,national,BG,Bulgaria,0,23370,7000039.0,333.855283,0.000000,TESSy


In [56]:
year_week = test['year_week'].str.split('-W')

year = [name[0] for name in year_week]
week = [name[1] for name in year_week]

In [59]:
test

,country,country_code,year_week,level,region,region_name,new_cases,tests_done,population,testing_rate,positivity_rate,testing_data_source,new_1,new_2,"(new_1, new_2)"
0,Austria,AT,2020-W15,national,AT,Austria,1954,12339,8858775.0,139.285624,15.835967,Manual webscraping,NaN,NaN,"[2020, 15]"
1,Austria,AT,2020-W16,national,AT,Austria,773,58488,8858775.0,660.226724,1.321639,Manual webscraping,NaN,NaN,"[2020, 16]"
2,Austria,AT,2020-W17,national,AT,Austria,479,33443,8858775.0,377.512692,1.432288,Manual webscraping,NaN,NaN,"[2020, 17]"
3,Austria,AT,2020-W18,national,AT,Austria,349,26598,8858775.0,300.244673,1.312129,Country website,NaN,NaN,"[2020, 18]"
4,Austria,AT,2020-W19,national,AT,Austria,249,42153,8858775.0,475.833284,0.590705,Country website,NaN,NaN,"[2020, 19]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3278,Sweden,SE,2020-W50,national,SE,Sweden,43639,270944,10230185.0,2648.476054,16.106280,TESSy,NaN,NaN,"[2020, 50]"
3279,Sweden,SE,2020-W51,national,SE,Sweden,46195,299447,10230185.0,2927.092716,15.426770,TESSy,NaN,NaN,"[2020, 51]"
3280,Sweden,SE,2020-W52,national,SE,Sweden,37147,232114,10230185.0,2268.913026,16.003774,TESSy,NaN,NaN,"[2020, 52]"
3281,Sweden,SE,2020-W53,national,SE,Sweden,41343,201011,10230185.0,1964.881378,20.567531,TESSy,NaN,NaN,"[2020, 53]"


In [40]:
dict(zip(test['country'], test['new_cases']))

{'Austria': 594,
 'Belgium': 14417,
 'Bulgaria': 5460,
 'Croatia': 5134,
 'Cyprus': 3376,
 'Czechia': 88740,
 'Denmark': 954,
 'Estonia': 4284,
 'Finland': 1671,
 'France': 128203,
 'Germany': 145511,
 'Greece': 4639,
 'Hungary': 14805,
 'Iceland': 144,
 'Ireland': 45726,
 'Italy': 11661,
 'Latvia': 6821,
 'Lithuania': 8339,
 'Luxembourg': 1065,
 'Malta': 1299,
 'Netherlands': 57026,
 'Norway': 36,
 'Poland': 8742,
 'Portugal': 387,
 'Romania': 32842,
 'Slovakia': 20970,
 'Slovenia': 13855,
 'Spain': 1738,
 'Sweden': 39485}